In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=9e27675b0b019d39286699e116fb81d7016580e5e5de803d6bb65e7328a0c68b
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


## Initiate the Spark session

In [3]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("myPySpark") \
    .getOrCreate()

In [4]:
# Check spark session
print(spark)

## Define the data structure

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([\
        StructField("link", StringType(), True),\
        StructField("headline", StringType(), True),\
        StructField("category", StringType(), True),\
        StructField("short_description", StringType(), True),\
        StructField("authors", StringType(), True),\
        StructField("date", StringType(), True),\
])

## Determine the folder location

In [10]:
df = spark.readStream.schema(schema).json("/content/input-05111940000063/")

In [11]:
df.printSchema()

root
 |-- link: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- category: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- date: string (nullable = true)



## Do the aggregation step

In [12]:
groupDF = df.select("category").groupBy("category").count()

groupDF.printSchema()

root
 |-- category: string (nullable = true)
 |-- count: long (nullable = false)



## Start the streaming task

In [ ]:
result = groupDF.writeStream\
  .format("csv")\
  .checkpoint("/checkpoint")
  .output("/output-05111940000063")
  .outputMode("append")\
  .start()

result.awaitTermination()